In [1]:
import os
import pandas as pd
import requests 
from AlphanumericsTeam.data.util import get_aug_oxford_df, filter_df_regions


REPO_ROOT = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, os.pardir, os.pardir))

{('Afghanistan', ''): 'AFG',
 ('Albania', ''): 'ALB',
 ('Algeria', ''): 'DZA',
 ('Andorra', ''): 'AND',
 ('Angola', ''): 'AGO',
 ('Argentina', ''): 'ARG',
 ('Aruba', ''): 'ABW',
 ('Australia', ''): 'AUS',
 ('Austria', ''): 'AUT',
 ('Azerbaijan', ''): 'AZE',
 ('Bahamas', ''): 'BHS',
 ('Bahrain', ''): 'BHR',
 ('Bangladesh', ''): 'BGD',
 ('Barbados', ''): 'BRB',
 ('Belarus', ''): 'BLR',
 ('Belgium', ''): 'BEL',
 ('Belize', ''): 'BLZ',
 ('Benin', ''): 'BEN',
 ('Bermuda', ''): 'BMU',
 ('Bhutan', ''): 'BTN',
 ('Bolivia', ''): 'BOL',
 ('Bosnia and Herzegovina', ''): 'BIH',
 ('Botswana', ''): 'BWA',
 ('Brazil', ''): 'BRA',
 ('Brunei', ''): 'BRN',
 ('Bulgaria', ''): 'BGR',
 ('Burkina Faso', ''): 'BFA',
 ('Burundi', ''): 'BDI',
 ('Cambodia', ''): 'KHM',
 ('Cameroon', ''): 'CMR',
 ('Canada', ''): 'CAN',
 ('Cape Verde', ''): 'CPV',
 ('Central African Republic', ''): 'CAF',
 ('Chad', ''): 'TCD',
 ('Chile', ''): 'CHL',
 ('China', ''): 'CHN',
 ('Colombia', ''): 'COL',
 ('Comoros', ''): 'COM',
 ('Cong

In [2]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']

In [3]:
EXAMPLE_INPUT_FILE = os.path.join(REPO_ROOT, "covid_xprize/validation/data/2020-09-30_historical_ip.csv")
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

prediction_input_df

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72605,Zimbabwe,NaN,2020-09-26,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
72606,Zimbabwe,NaN,2020-09-27,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
72607,Zimbabwe,NaN,2020-09-28,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,3.0
72608,Zimbabwe,NaN,2020-09-29,2.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0


In [4]:
EXAMPLE_OUTPUT_FILE = os.path.join(REPO_ROOT, "2020-08-01_2020-08-04_predictions_example.csv")
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")                           

In [5]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

# Has 6 additional columns 
# 'New Cases' 
# 'GeoID' 
# 'Holidays' 
# 'pop_2020' 
# 'area_km2' 
# 'density_perkm2'
df = get_aug_oxford_df() 
df = filter_df_regions(df)
assert df.CountryName.unique().size == 180
assert df.RegionName.unique().size == 56 + 1 

In [6]:
df.sample(3)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,NewCases,GeoID,Holidays,pop_2020,area_km2,density_perkm2
89455,United States,USA,New Jersey,US_NJ,STATE_TOTAL,2020-07-24,3.0,0.0,2.0,1.0,...,62.18,62.18,87.5,87.5,458.0,United States__New Jersey,0.0,8936570.0,19047.0,469.0
72368,Syria,SYR,,NaN,NAT_TOTAL,2020-09-11,3.0,1.0,2.0,1.0,...,39.42,39.42,0.0,0.0,60.0,Syria__,1.0,17500658.0,185180.0,95.0
62518,Poland,POL,,NaN,NAT_TOTAL,2020-02-13,0.0,NaN,0.0,NaN,...,3.85,3.85,0.0,0.0,0.0,Poland__,0.0,37846611.0,312679.0,121.0


In [7]:
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)

In [8]:
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]

In [9]:
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
79963,United States,California,2020-12-18,1802195.0,48221.0
79964,United States,California,2020-12-19,1842557.0,40362.0
79965,United States,California,2020-12-20,1884033.0,41476.0
79966,United States,California,2020-12-21,1923887.0,39854.0
79967,United States,California,2020-12-22,NaN,0.0


In [12]:
# Reload the module to get the latest changes
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

DATA_PATH = os.path.join("data", 'OxCGRT_latest_aug.csv')
predictor = XPrizePredictor(None, DATA_PATH)
 
predictor_model = predictor.train()

model_weights_file = "models/trained_model_weights.h5"
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights(model_weights_file)

Creating numpy arrays for Keras for each country...
Numpy arrays created
Trial 0
Epoch 1/5
167/167 [==============================] - 1s 6ms/step - loss: 0.9230 - val_loss: 0.1163
Epoch 2/5
167/167 [==============================] - 1s 3ms/step - loss: 0.0738 - val_loss: 0.0688
Epoch 3/5
167/167 [==============================] - 1s 3ms/step - loss: 0.0696 - val_loss: 0.0682
Epoch 4/5
167/167 [==============================] - 1s 3ms/step - loss: 0.0715 - val_loss: 0.0728
Epoch 5/5
186/186 [==============================] - 0s 2ms/step - loss: 0.0695
Train Loss: 0.06963951885700226
Val Loss: 0.06819598376750946
Test Loss: 0.06954781711101532
Trial 1
Epoch 1/5
167/167 [==============================] - 1s 6ms/step - loss: 1.0440 - val_loss: 1.0508
Epoch 2/5
167/167 [==============================] - 1s 3ms/step - loss: 0.3348 - val_loss: 0.0784
Epoch 3/5
167/167 [==============================] - 1s 3ms/step - loss: 0.0689 - val_loss: 0.0777
Epoch 4/5
167/167 [==========================

In [15]:
predictor = XPrizePredictor(model_weights_file, DATA_PATH)

NPIS_INPUT_FILE = "../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"

preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)
preds_df.head()

KeyError: "['density_perkm2', 'Holidays'] not in index"

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv
!head predictions/2020-08-01_2020-08-04.csv

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [ ]:
# Check the pediction file is valid
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")